In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Jul_dict_JSON.json', 'r') as json_file:
    link_day_minute_Jul_dict_JSON = json.load(json_file)

In [11]:
week_day_Jul_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30, 31]
#week_day_Jul_list = [2, 9, 16, 23, 30]

In [12]:
link_day_minute_Jul_list = []
for link_idx in range(24):
    for day in week_day_Jul_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Jul_list.append(link_day_minute_Jul_dict_JSON[key] ['PM_flow_minute'][minute_idx])

In [13]:
len(link_day_minute_Jul_list)

63360

In [14]:
x = np.matrix(link_day_minute_Jul_list)
x = np.matrix.reshape(x, 24, 2640)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [15]:
np.size(x, 0), np.size(x, 1)

(24, 2640)

In [16]:
x[:,:2]

matrix([[  4720.00071095,   3100.27391698],
        [  2161.24531072,   2661.45999365],
        [  3682.67741879,   4208.65840104],
        [  5194.35658607,   4730.62857294],
        [  4573.09361897,   4322.41614521],
        [  3221.91848896,   1923.282846  ],
        [  2752.2583681 ,   2551.6043617 ],
        [  2966.27422946,   4172.52298046],
        [ 11229.22412555,  10254.43462874],
        [  6265.23773967,   8857.20794618],
        [ 14992.6164319 ,  15443.65795142],
        [  5630.58999872,   5275.80607693],
        [  3084.27369578,   1896.94854445],
        [  3310.3345766 ,   1643.36468926],
        [  3930.91403416,   2752.79628201],
        [  2661.34241823,   1899.00605027],
        [  1555.34068427,   2547.98846192],
        [  2830.42313638,   3866.60188706],
        [  2736.16094686,   2692.27605279],
        [  7570.24907376,   8205.13340514],
        [ 10558.79114216,   8494.25636705],
        [ 10567.88798277,  10815.86736943],
        [  5926.75993551,   2098

In [17]:
np.size(A,0), np.size(A,1)

(24, 140)

In [18]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b' * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [8e-02, 3e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.80462147e+06  0.00000000e+00  4.49e+03 2.41e+02  9.20e+05     0s
   1   8.05395124e+05 -2.17699851e+06  7.53e+02 1.52e+02  2.09e+05     0s
   2   4.34553443e+05 -1.09586597e+06  7.53e-04 1.14e-13  2.73e+04     0s
   3  -9.95346090e+04 -3.08644926e+05  7.53e-10 5.68e-14  3.7

In [19]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Jul_weekday_PM.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [20]:
lam_list

[2.6345339860918783e-09,
 6668.481500814973,
 2.928315951587706e-09,
 193.31657858277595,
 3430.14648245008,
 1.5551429316610488e-06,
 5.641035728977525e-07,
 9.867312918222879e-05,
 1.181344247512724e-08,
 6180.08129923317,
 1.1813442463195616e-08,
 1.1813442470354739e-08,
 1.1724379734147336e-08,
 1.17916120421446e-08,
 6075.425897317694,
 1215.9407120264586,
 0.7042741739928395,
 8593.510043313747,
 8171.489476738776,
 4976.757167387854,
 1.5556026900164996e-06,
 1613.237965715643,
 4782.578253862286,
 671.3488494169596,
 1389.6887577756406,
 939.6991365260028,
 8.916059893212488e-07,
 1986.9859496369147,
 0.26063418299503116,
 0.30578747386506855,
 1809.5725906600524,
 7548.036288637269,
 5793.702733383603,
 8.826690607135152e-07,
 8.831172649072892e-07,
 3.778784537184363e-07,
 3.778784084276693e-07,
 3.794409279743039e-07,
 0.8136343689901011,
 5341.12484741988,
 8087.94639947922,
 1043.210855178886,
 8.105891366854545e-07,
 8.105889253313642e-07,
 14408.669854546124,
 4.64859595